## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-01-06-58-10 +0000,nypost,‘Life After Lockup’ star accused of assaulting...,https://nypost.com/2026/01/01/us-news/life-aft...
1,2026-01-01-06-53-48 +0000,nypost,Mayor Zohran Mamdani names DOT veteran Mike Fl...,https://nypost.com/2026/01/01/us-news/mayor-zo...
2,2026-01-01-06-43-42 +0000,nypost,Buddhist monks — and their pup — persist in pe...,https://nypost.com/2026/01/01/us-news/buddhist...
3,2026-01-01-06-43-29 +0000,bbc,Starmer vows to 'defeat decline and division' ...,https://www.bbc.com/news/articles/c98p5p6rmm2o...
4,2026-01-01-06-37-27 +0000,bbc,Zohran Mamdani sworn in as first Muslim New Yo...,https://www.bbc.com/news/videos/c4glvg5yrlyo?a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
119,trump,33
39,year,31
38,new,28
123,guard,12
48,world,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
35,2026-01-01-02-19-53 +0000,nypost,"Trump calls for ‘peace on Earth,’ vows to ‘get...",https://nypost.com/2025/12/31/us-news/trump-ca...,125
86,2025-12-31-23-02-41 +0000,nyt,"In New Year’s Speech to Russia, Putin Says Lit...",https://www.nytimes.com/2025/12/31/world/europ...,91
13,2026-01-01-05-09-00 +0000,nypost,Thousands of New Year’s Eve revelers brave fri...,https://nypost.com/2026/01/01/us-news/new-year...,84
137,2025-12-31-19-02-59 +0000,nypost,NJ drivers hit with 9% state gas tax hike to k...,https://nypost.com/2025/12/31/us-news/nj-drive...,83
197,2025-12-31-13-14-45 +0000,wapo,"In Russia, plans to cut mobile internet on New...",https://www.washingtonpost.com/world/2025/12/3...,80


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
35,125,2026-01-01-02-19-53 +0000,nypost,"Trump calls for ‘peace on Earth,’ vows to ‘get...",https://nypost.com/2025/12/31/us-news/trump-ca...
112,45,2025-12-31-21-20-43 +0000,nypost,"Trump pulling National Guard from Chicago, Por...",https://nypost.com/2025/12/31/us-news/trump-re...
74,34,2025-12-31-23-27-34 +0000,nypost,Accused Christmas killer hit with rare first-d...,https://nypost.com/2025/12/31/us-news/christma...
138,34,2025-12-31-19-02-13 +0000,nypost,DOJ ‘working around the clock’ on Epstein file...,https://nypost.com/2025/12/31/us-news/doj-work...
28,33,2026-01-01-03-28-50 +0000,cbc,U.S. military says 8 killed in strikes on 5 mo...,https://www.cbc.ca/news/world/us-venezuela-dru...
61,32,2026-01-01-00-11-00 +0000,wsj,Trump’s surprise veto of a Colorado water proj...,https://www.wsj.com/politics/policy/trump-veto...
187,32,2025-12-31-14-48-14 +0000,nypost,"Russian drone attack injures 6, including chil...",https://nypost.com/2025/12/31/world-news/russi...
143,32,2025-12-31-19-00-05 +0000,nypost,San Fran mayor signs bill that could give blac...,https://nypost.com/2025/12/31/us-news/san-fran...
69,31,2025-12-31-23-37-45 +0000,nypost,Tax hike on NY workers will be needed to fund ...,https://nypost.com/2025/12/31/us-news/mamdani-...
106,30,2025-12-31-21-46-20 +0000,nypost,Socialist LA official claims she won’t accept ...,https://nypost.com/2025/12/31/us-news/socialis...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
